In [0]:
import tensorflow as tf

In [36]:
class BuildModel:
    INP_SIZE = 224
    LEARNING_RATE = 0.0001

    def __init__(self, is_fine_tuning=False) -> None:
        self.is_fine_tuning = is_fine_tuning
        basemodel = self.build_base(self.is_fine_tuning)
        basemodel.trainable = is_fine_tuning
        self.model = self.build_seq(basemodel)
        optimizer = self.get_optimizer()
        self.compile(optimizer=optimizer)

        print('The model', self.get_name(), 'initialised...')

    def build_seq(self, basemodel):
        return tf.keras.Sequential(
            [basemodel,
             tf.keras.layers.GlobalAveragePooling2D(),
             tf.keras.layers.Dense(101, activation='softmax'),
             ])

    def get_optimizer(self):
        return tf.keras.optimizers.Adam(learning_rate=self.LEARNING_RATE)

    def compile(self, optimizer):
        model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    def get_tensorboard_callback(self):
        return tf.keras.callbacks.TensorBoard(
        log_dir='logs/' + self.get_name(),
        write_graph=False, update_freq=100, profile_batch=0)

    def get_name(self):
        name = self.__class__.__name__
        if not self.is_fine_tuning:
            name += '_frozen'
        return name

    def build_base(self, is_fine_tuning):
        return None


class MobilNet(BuildModel):
    def build_base(self, is_fine_tuning):
        model = tf.keras.applications.MobileNetV2(
            input_shape=(self.INP_SIZE, self.INP_SIZE, 3),
            include_top=False,
            weights='imagenet')
        return model


class EfficientNet(BuildModel):
    def build_base(self, is_fine_tuning):
        model = tf.keras.applications.EfficientNetV2S(
            input_shape=(self.INP_SIZE, self.INP_SIZE, 3),
            include_top=False,
            weights='imagenet',
            include_preprocessing=False, )
        return model


In [37]:
model = MobilNet(is_fine_tuning=True).model

The model MobilNet initialised...


In [38]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_8   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 101)               129381    
                                                                 
Total params: 2,387,365
Trainable params: 2,353,253
Non-trainable params: 34,112
_________________________________________________________________
